In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import os
import scipy.linalg

from graph_utils import graph_gen
from problem import ElecVehCharg
from plot_utils import MyFigure

from algorithms import UDC
# Set up the logger to print info messages for understandability.
import logging
import sys
logging.basicConfig(level=logging.INFO, stream=sys.stdout, format='')


In [12]:
# generate problem

# network

num_node = 50
num_edge_list = [55, 60, 70, 90, 120, 200, 500, 1000]

# graph_gen(num_node, num_edge, B=1, ring=True)
for num_edge in num_edge_list:
    graph_gen(num_node, num_edge, B=1)

net_dir = f'instance/graph/N{num_node}E{num_edge}'
network = np.load(f'{net_dir}/subgraph_W.npy')
print(network.shape)
# print(network)


Graph with 50 nodes and 55 edges


c:\Users\xtq\OneDrive - shanghaitech.edu.cn\Research\Dual AMM\numerical\exp2_elec_veh_charg\graph_utils.py:86: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`). Consider using `matplotlib.pyplot.close()`.
  plt.figure()


Graph with 50 nodes and 60 edges
Graph with 50 nodes and 70 edges
Graph with 50 nodes and 90 edges
Graph with 50 nodes and 120 edges
Graph with 50 nodes and 200 edges
Graph with 50 nodes and 500 edges
Graph with 50 nodes and 1000 edges
(50, 50)


In [ ]:
# problem data
num_node = 50
prob = ElecVehCharg(num_node)
# prob = Synthetic(parameters, debug=True)

prob.gen()
prob.load()
print(f'x* {prob.x_star}')
print(f'||x*|| {np.linalg.norm(prob.x_star)}\n')

In [17]:
# load instance 
num_node = 50
instance_name = 'instance1'
instance_dir = 'instance/' + instance_name

# network
num_edge = 120
network = np.load(f'instance/graph/N50E{num_edge}/subgraph_W.npy')
print(network.shape)

# problem data
prob = ElecVehCharg(num_node)
prob.save_dir = instance_dir + '/problem'
prob.load()

print(f'x* {prob.x_star}')
print(f'||x*|| {np.linalg.norm(prob.x_star)}\n')


(50, 50)
loading a ElecVehCharg problem, N=50
problem loaded:
c (50,)
d (50,)
b (1,)
x_star (50,)
opt_val 1.8027609655212484
x* [ 9.81e-01 -2.02e-12  1.37e-11  4.92e-11  5.80e-12  1.00e+00  3.74e-12
  1.68e-12  6.11e-12  7.12e-12  1.47e-11 -7.63e-13  1.00e+00 -4.76e-12
  1.90e-11  3.56e-12  5.18e-02  1.38e-12  4.42e-13 -7.67e-13  1.08e-12
  5.26e-12  6.20e-01 -4.55e-12  7.67e-12  8.38e-02 -1.32e-12  1.00e+00
 -3.66e-12  1.00e+00  9.58e-01  6.76e-12  3.63e-01  1.35e-11  1.31e-12
  1.00e+00  8.03e-11  4.12e-12  1.80e-11  2.50e-11  5.83e-11 -2.84e-12
  7.08e-11  2.07e-11 -1.14e-12  1.00e+00  1.40e-11 -1.47e-12  4.70e-11
  5.91e-01]
||x*|| 2.958778115102063



In [51]:
# UDC

# best parameters (for 120 edge graph):
# PEXTRA: r1
# PGC: r0.1
# ALT: 2
# D.A.: 0.5

# MAX_ITER = 500
MAX_ITER = 1000
# MAX_ITER = 2000

log_dir = 'log'
if not os.path.exists(log_dir):
    os.makedirs(log_dir)

num_edge_list = [50, 60, 90, 120, 200, 500, 1000]
# num_edge = 120

# rho_choice = [0.1, 1, 10, 100]
# rho_choice = [0.5, 1, 2, 4]
# rho_choice = [0.25, 0.5]
rho_choice = [1]
# rho_choice = [0.1]

color = ['b', 'g', 'r', 'c', 'm', 'y', 'k']
linestyle = ['-', '--', '-.', ':', '']
al_list = [0.1*i for i in range(10, 0, -1)]

obj_err_figure = MyFigure(filename='obj_err', 
                            xlabel=r'$\mathrm{iteration}$ $k$', 
                            ylabel='objective error',
                            yscale='log')
cons_vio_figure = MyFigure(filename='cons_vio', 
                            xlabel=r'$\mathrm{iteration}$ $k$', 
                            ylabel='constraint violation',
                            yscale='log')
x_dis_figure = MyFigure(filename='x_distance', 
                            xlabel=r'$\mathrm{iteration}$ $k$', 
                            ylabel='x distance',
                            yscale='log')
    
np.set_printoptions(formatter={'float':lambda x: f' {x:.2e}' if x>0 else f'{x:.2e}'})

for num_edge in num_edge_list:
# for rho in rho_choice:
    
    ci = 1
    ali = num_edge_list.index(num_edge)
    # ci = rho_choice.index(rho)
    # ali = 1
    
    c = color[ci]
    al = al_list[ali]
    
    network = np.load(f'instance/graph/N50E{num_edge}/subgraph_W.npy')
    
    rho = 1
    if True:
    # if alpha != 0:
    # if rho != 1:
    # if rho == 4:
        alg = UDC(prob, network, rho=rho, alpha=0,
                    param_setting='PEXTRA', verbose=False)
        # alg = UDC(prob, network, rho=rho, alpha=alpha,
        #           param_setting='PGC', verbose=False)
        # alg = UDC(prob, network, rho=rho, alpha=alpha,
        #           param_setting='DPGA', verbose=False)
        # alg = UDC(prob, network, rho=rho, alpha=alpha,
        #           param_setting='DistADMM', verbose=False)
        # alg = UDC(prob, network, rho=rho, alpha=0,
        #         param_setting='ALT', verbose=False)
        for i in range(MAX_ITER):
            alg.step()
        
    prefix = f'{alg.name}_a{0}_r{rho}'
    filename_oe = f'log/N{num_node}/{prefix}_oe.txt'
    filename_cv = f'log/N{num_node}/{prefix}_cv.txt'
    filename_xd = f'log/N{num_node}/{prefix}_xd.txt'
    
    obj_err_figure.add_line_file(f'PEXTRA_r{rho}_e{num_edge}', filename_oe, style=c, alpha=al)
    cons_vio_figure.add_line_file(f'PEXTRA_r{rho}_e{num_edge}', filename_cv, style=c, alpha=al)
    x_dis_figure.add_line_file(f'PEXTRA_r{rho}_e{num_edge}', filename_xd, style=c, alpha=al)
        
    obj_err_figure.paint(MAX_ITER=MAX_ITER)
    cons_vio_figure.paint(MAX_ITER=MAX_ITER, nonnegy=True)
    x_dis_figure.paint(MAX_ITER=MAX_ITER, nonnegy=True)
    
    
    
    # c = color[2]
    # rho =0.1
    # if True:
    # # if alpha != 0:
    # # if rho != 1:
    # # if rho == 4:
    #     # alg = UDC(prob, network, rho=rho, alpha=0,
    #     #           param_setting='PEXTRA', verbose=False)
    #     alg = UDC(prob, network, rho=rho, alpha=0,
    #                 param_setting='PGC', verbose=False)
    #     # alg = UDC(prob, network, rho=rho, alpha=alpha,
    #     #           param_setting='DPGA', verbose=False)
    #     # alg = UDC(prob, network, rho=rho, alpha=alpha,
    #     #           param_setting='DistADMM', verbose=False)
    #     # alg = UDC(prob, network, rho=rho, alpha=alpha,
    #     #         param_setting='ALT', verbose=False)
    #     for i in range(MAX_ITER):
    #         alg.step()
        
    # prefix = f'{alg.name}_a{0}_r{rho}'
    # filename_oe = f'log/N{num_node}/{prefix}_oe.txt'
    # filename_cv = f'log/N{num_node}/{prefix}_cv.txt'
    # filename_xd = f'log/N{num_node}/{prefix}_xd.txt'
    
    # obj_err_figure.add_line_file(f'PGC_r{rho}_e{num_edge}', filename_oe, style=c, alpha=al)
    # cons_vio_figure.add_line_file(f'PGC_r{rho}_e{num_edge}', filename_cv, style=c, alpha=al)
    # x_dis_figure.add_line_file(f'PGC_r{rho}_e{num_edge}', filename_xd, style=c, alpha=al)
        
    # obj_err_figure.paint(MAX_ITER=MAX_ITER)
    # cons_vio_figure.paint(MAX_ITER=MAX_ITER, nonnegy=True)
    # x_dis_figure.paint(MAX_ITER=MAX_ITER, nonnegy=True)

    c = color[3]
    rho =0.5
    if True:
    # if alpha != 0:
    # if rho != 1:
    # if rho == 4:
        # alg = UDC(prob, network, rho=rho, alpha=0,
        #           param_setting='PEXTRA', verbose=False)
        # alg = UDC(prob, network, rho=rho, alpha=alpha,
        #           param_setting='PGC', verbose=False)
        # alg = UDC(prob, network, rho=rho, alpha=alpha,
        #           param_setting='DPGA', verbose=False)
        alg = UDC(prob, network, rho=0.5, alpha=0,
                    param_setting='DistADMM', verbose=False)
        # alg = UDC(prob, network, rho=rho, alpha=alpha,
        #         param_setting='ALT', verbose=False)
        for i in range(MAX_ITER):
            alg.step()
        
    prefix = f'{alg.name}_a{0}_r{rho}'
    filename_oe = f'log/N{num_node}/{prefix}_oe.txt'
    filename_cv = f'log/N{num_node}/{prefix}_cv.txt'
    filename_xd = f'log/N{num_node}/{prefix}_xd.txt'
    
    obj_err_figure.add_line_file(f'D.A._r{rho}_e{num_edge}', filename_oe, style=c, alpha=al)
    cons_vio_figure.add_line_file(f'D.A._r{rho}_e{num_edge}', filename_cv, style=c, alpha=al)
    x_dis_figure.add_line_file(f'D.A._r{rho}_e{num_edge}', filename_xd, style=c, alpha=al)
        
    obj_err_figure.paint(MAX_ITER=MAX_ITER)
    cons_vio_figure.paint(MAX_ITER=MAX_ITER, nonnegy=True)
    x_dis_figure.paint(MAX_ITER=MAX_ITER, nonnegy=True)
    
    
    c = color[4]
    rho =2
    if True:
    # if alpha != 0:
    # if rho != 1:
    # if rho == 4:
        # alg = UDC(prob, network, rho=rho, alpha=0,
        #           param_setting='PEXTRA', verbose=False)
        # alg = UDC(prob, network, rho=rho, alpha=alpha,
        #           param_setting='PGC', verbose=False)
        # alg = UDC(prob, network, rho=rho, alpha=alpha,
        #           param_setting='DPGA', verbose=False)
        # alg = UDC(prob, network, rho=0.5, alpha=0,
        #             param_setting='DistADMM', verbose=False)
        alg = UDC(prob, network, rho=rho, alpha=0,
                param_setting='ALT', verbose=False)
        for i in range(MAX_ITER):
            alg.step()
        
    prefix = f'{alg.name}_a{0}_r{rho}'
    filename_oe = f'log/N{num_node}/{prefix}_oe.txt'
    filename_cv = f'log/N{num_node}/{prefix}_cv.txt'
    filename_xd = f'log/N{num_node}/{prefix}_xd.txt'
    
    obj_err_figure.add_line_file(f'ALT_r{rho}_e{num_edge}', filename_oe, style=c, alpha=al)
    cons_vio_figure.add_line_file(f'ALT_r{rho}_e{num_edge}', filename_cv, style=c, alpha=al)
    x_dis_figure.add_line_file(f'ALT_r{rho}_e{num_edge}', filename_xd, style=c, alpha=al)
        
    obj_err_figure.paint(MAX_ITER=MAX_ITER)
    cons_vio_figure.paint(MAX_ITER=MAX_ITER, nonnegy=True)
    x_dis_figure.paint(MAX_ITER=MAX_ITER, nonnegy=True)

UDC setting: PEXTRA
d: (50,)
reset
UDC_PEXTRA alpha 0 rho 1, iter 0, obj err: 1.80e+00, cons vio: 5.00e+00
time 0.00, saved

UDC_PEXTRA alpha 0 rho 1, iter 100, obj err: 2.78e-02, cons vio: 0.00e+00
time 0.59, saved

UDC_PEXTRA alpha 0 rho 1, iter 200, obj err: 1.40e-03, cons vio: 2.73e-03
time 1.17, saved

UDC_PEXTRA alpha 0 rho 1, iter 300, obj err: 8.15e-04, cons vio: 0.00e+00
time 1.78, saved

UDC_PEXTRA alpha 0 rho 1, iter 400, obj err: 9.48e-05, cons vio: 1.64e-04
time 2.34, saved

UDC_PEXTRA alpha 0 rho 1, iter 500, obj err: 6.91e-05, cons vio: 1.18e-04
time 2.91, saved

UDC_PEXTRA alpha 0 rho 1, iter 600, obj err: 2.00e-05, cons vio: 0.00e+00
time 3.54, saved

UDC_PEXTRA alpha 0 rho 1, iter 700, obj err: 4.09e-06, cons vio: 6.98e-06
time 4.11, saved

UDC_PEXTRA alpha 0 rho 1, iter 800, obj err: 5.09e-07, cons vio: 8.68e-07
time 4.69, saved

UDC_PEXTRA alpha 0 rho 1, iter 900, obj err: 8.62e-08, cons vio: 0.00e+00
time 5.30, saved

UDC_PEXTRA alpha 0 rho 1, iter 1000, obj err: 1

In [6]:
# DPMM
# instance 1: a0.1_r5_t1.5
# instance 2: a0.1_r1_t1.5

from plot_utils import MyFigure

alg_name = 'DPMM'
MAX_ITER = 2000
# MAX_ITER = 500

# theta_choice = [0.5, 1, 1.5, 1.9]
# theta_choice = [0.1, 1.9]
theta_choice = [1.5]

# rho_choice = [0.1, 1, 10]
# rho_choice = [0.5, 1, 2, 4, 5]
rho_choice = [5]

# alpha_choice = [0, 10, 100]
# alpha_choice = [0.1, 1, 10]
alpha_choice = [0.1]

color = ['b', 'g', 'r', 'c', 'm', 'y', 'k']
linestyle = ['-', '--', '-.', ':', '']

obj_err_figure = MyFigure(filename='obj_err', 
                            xlabel=r'$\mathrm{iteration}$ $k$', 
                            ylabel='objective error',
                            yscale='log')
cons_vio_figure = MyFigure(filename='cons_vio', 
                            xlabel=r'$\mathrm{iteration}$ $k$', 
                            ylabel='constraint violation',
                            yscale='log')
x_dis_figure = MyFigure(filename='x_distance', 
                            xlabel=r'$\mathrm{iteration}$ $k$', 
                            ylabel='x distance',
                            yscale='log')


for alpha in alpha_choice:
    for rho in rho_choice:
        # ci = alpha_choice.index(alpha)
        # lsi = rho_choice.index(rho)
        ci = rho_choice.index(rho)
        lsi = alpha_choice.index(alpha)
        c = color[ci]
        ls = linestyle[lsi]
        
        for theta in theta_choice:
            
            if False:
            # if rho != 1:
            # if rho == 5:
            # if alpha != 0:
            # if theta == 1.9:
                alg = UDC(prob, network, rho=rho, alpha=alpha, theta=theta,
                            param_setting='DPMM', verbose=False)
                for i in range(MAX_ITER):
                    alg.step()

            # plot
            # ci = theta_choice.index(theta)
            # c = color[ci]
            
            prefix = f'{alg_name}_a{alpha}_r{rho}_t{theta}'
            filename_oe = f'log/N{num_node}/{prefix}_oe.txt'
            filename_cv = f'log/N{num_node}/{prefix}_cv.txt'
            filename_xd = f'log/N{num_node}/{prefix}_xd.txt'
            
            obj_err_figure.add_line_file(prefix, filename_oe, style=c+ls)
            cons_vio_figure.add_line_file(prefix, filename_cv, style=c+ls)
            x_dis_figure.add_line_file(prefix, filename_xd, style=c+ls)
                
            obj_err_figure.paint(MAX_ITER=MAX_ITER)
            cons_vio_figure.paint(MAX_ITER=MAX_ITER, nonnegy=True)
            x_dis_figure.paint(MAX_ITER=MAX_ITER, nonnegy=True)


c:\Users\xtq\OneDrive - shanghaitech.edu.cn\Research\Dual AMM\numerical\exp3_synthetic_IPLUX\plot_utils.py:131: UserWarning: Data has no positive values, and therefore cannot be log-scaled.
  fig.savefig(f'{self.filename}.png', bbox_inches='tight', transparent=False)


In [ ]:
# dual subgradient
#
# instance 1: g5
# instance 2: g10

from plot_utils import MyFigure

alg_name = 'dual_subgradient'
MAX_ITER = 500
# gamma_choice = [0.1, 1, 5, 25]
# gamma_choice = [2.5, 5, 10, 50]
gamma_choice = [2.5, 5, 10]
color = ['b', 'g', 'r', 'c', 'm', 'y', 'k']

obj_err_figure = MyFigure(filename='obj_err', 
                            xlabel=r'$\mathrm{iteration}$ $k$', 
                            ylabel='objective error',
                            yscale='log')
cons_vio_figure = MyFigure(filename='cons_vio', 
                            xlabel=r'$\mathrm{iteration}$ $k$', 
                            ylabel='constraint violation',
                            yscale='log')
x_dis_figure = MyFigure(filename='x_distance', 
                            xlabel=r'$\mathrm{iteration}$ $k$', 
                            ylabel='x distance',
                            yscale='log')


for gamma in gamma_choice:
    if True:
    # if gamma == 50:
        alg = dual_subgradient(prob, network, gamma)
        for i in range(MAX_ITER):
            alg.step()

    # plot
    ci = gamma_choice.index(gamma)
    c = color[ci]
    
    prefix = f'{alg_name}_g{gamma}'
    filename_oe = f'log/N{num_node}/{prefix}_oe.txt'
    filename_cv = f'log/N{num_node}/{prefix}_cv.txt'
    filename_xd = f'log/N{num_node}/{prefix}_xd.txt'
    
    obj_err_figure.add_line_file(prefix, filename_oe, style=c)
    cons_vio_figure.add_line_file(prefix, filename_cv, style=c)
    x_dis_figure.add_line_file(prefix, filename_xd, style=c)
        
    obj_err_figure.paint(MAX_ITER=MAX_ITER)
    cons_vio_figure.paint(MAX_ITER=MAX_ITER, nonnegy=True)
    x_dis_figure.paint(MAX_ITER=MAX_ITER, nonnegy=True)

    

In [50]:
# compare fine-tuned algorithms

# instance 1
# UDC_PEXTRA: r4
# UDC_PGC: r0.4
# UDC_DPGA: r2
# UDC_DistADMM: r1
# ALT: r2
# DPMM: a0.1 r5 t1.5
# IPLUX: a5.0 r5
# dual subgradient: g5

alg_list = []
alg_list.append(UDC(prob, network, alpha=0, rho=4, param_setting='PEXTRA'))
alg_list.append(UDC(prob, network, alpha=0, rho=0.4, param_setting='PGC'))
alg_list.append(UDC(prob, network, alpha=0, rho=2, param_setting='DPGA'))
alg_list.append(UDC(prob, network, alpha=0, rho=1, param_setting='DistADMM'))
alg_list.append(UDC(prob, network, alpha=0, rho=2, param_setting='ALT'))
alg_list.append(UDC(prob, network, alpha=0.1, rho=5, theta=1.5, 
                                                    param_setting='DPMM'))
alg_list.append(IPLUX(prob, network, alpha=5.0, rho=5))
alg_list.append(dual_subgradient(prob, network, gamma=5))
oe_logs = dict()
cv_logs = dict()
xd_logs = dict()


# instance 2
# param_best_list = [['IPLUX', 4.0, 1], 
#                  ['UDC_pt', 20, 4], 
#                  ['UDC_PEXTRA', 0.1, 1], 
#                  ['UDC_DistADMM', 0.1, 2],
#                  ['DPMM', 0.1, 1, 1.5],
#                  ['dual_subgradient', 10]]
# alg_list = []
# alg_list.append(IPLUX(prob, network, alpha=4.0, rho=1))
# alg_list.append(UDC(prob, network, alpha=20, rho=4, param_setting='pt'))
# alg_list.append(UDC(prob, network, alpha=0.1, rho=1, param_setting='PEXTRA'))
# alg_list.append(UDC(prob, network, alpha=0.1, rho=2, param_setting='DistADMM'))
# alg_list.append(UDC(prob, network, alpha=0.1, rho=1, theta=1.5, param_setting='DPMM'))
# alg_list.append(dual_subgradient(prob, network, gamma=10))




color = ['tab:green', 'chocolate', 'red', 'chartreuse', 'navy', 'grey', 'k', 'crimson']
marker = ['.', 'o', 'v', '^', '8', 's', '*', 'd', 'X']
MAX_ITER = 2000

obj_err_figure = MyFigure(filename='compare_obj_err', 
                            xlabel=r'$\mathrm{Iteration}$ $k$', 
                            ylabel=r'$\mathrm{Objective}\ \mathrm{Error}$')

cons_vio_figure = MyFigure(filename='compare_cons_vio', 
                            xlabel=r'$\mathrm{Iteration}$ $k$', 
                            ylabel=r'$\mathrm{Constraint}\ \mathrm{Violation}$')

x_dis_figure = MyFigure(filename='compare_x_distance', 
                            xlabel=r'$\mathrm{Iteration}$ $k$', 
                            ylabel=r'$\Vert \mathbf{x}^k - \mathbf{x}^\star \Vert$')

for alg in alg_list:
    ci = alg_list.index(alg)
    c = color[ci]
    m = marker[ci]
    # for i in range(MAX_ITER):
    #     alg.step()
    # log_dir = 'log/N20'
    # filename_oe = f'{log_dir}/{alg.file_prefix}_oe.txt'
    # filename_cv = f'{log_dir}/{alg.file_prefix}_cv.txt'
    # filename_xd = f'{log_dir}/{alg.file_prefix}_xd.txt'
    
    # instance 1
    log_dir = f'instance/instance1_N20E40/log'
    file_alg_name = alg.name.replace(' ', '_').replace('-', '_')
    filename_oe = f'{log_dir}/{file_alg_name}/all_log/{alg.file_prefix}_oe.txt'
    filename_cv = f'{log_dir}/{file_alg_name}/all_log/{alg.file_prefix}_cv.txt'
    filename_xd = f'{log_dir}/{file_alg_name}/all_log/{alg.file_prefix}_xd.txt'
    oe_logs[alg.name] = np.loadtxt(filename_oe)
    cv_logs[alg.name] = np.loadtxt(filename_cv)
    xd_logs[alg.name] = np.loadtxt(filename_xd)
    
    # obj_err_figure.add_line_file(alg.name, filename_oe, style=c, marker=m)
    # cons_vio_figure.add_line_file(alg.name, filename_cv, style=c, marker=m)
    # x_dis_figure.add_line_file(alg.name, filename_xd, style=c, marker=m)

    # obj_err_figure.paint(MAX_ITER=MAX_ITER)
    # cons_vio_figure.paint(MAX_ITER=MAX_ITER, nonnegy=True)
    # x_dis_figure.paint(MAX_ITER=MAX_ITER, nonnegy=True)


UDC setting: PEXTRA
self.prob minimize quad_over_lin(param1429 @ var1428, 1.0) + param1430 @ var1428 + param1431[0] @ norm1(var1428) + 0.0 @ quad_over_lin(Promote(param1432[0], (3,)) @ var1428 + -param1433, 1.0) + 0.5 @ quad_over_lin(maximum(param1434 + quad_over_lin(var1428 + -param1436, 1.0) + -param1437, 0.0), 1.0) + 0.5 @ quad_over_lin(param1435 + param1440 @ var1428, 1.0)
subject to QuadForm(var1428 + -param1438, [[1.00 0.00 0.00]
 [0.00 1.00 0.00]
 [0.00 0.00 1.00]]) <= param1439[0]
reset
UDC_PEXTRA alpha 0 rho 4, iter 0, obj err: 1.79e+00, cons vio: 0.00e+00
time 0.02, saved

UDC setting: PGC
self.prob minimize quad_over_lin(param1481 @ var1480, 1.0) + param1482 @ var1480 + param1483[0] @ norm1(var1480) + 0.0 @ quad_over_lin(Promote(param1484[0], (3,)) @ var1480 + -param1485, 1.0) + 0.5 @ quad_over_lin(maximum(param1486 + quad_over_lin(var1480 + -param1488, 1.0) + -param1489, 0.0), 1.0) + 0.5 @ quad_over_lin(param1487 + param1492 @ var1480, 1.0)
subject to QuadForm(var1480 + -pa

In [90]:

import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib import rc, font_manager
from matplotlib.ticker import FormatStrFormatter, LogFormatter
from matplotlib.legend_handler import HandlerTuple
# rc('mathtext', fontset='stixsans')
# rc('text', usetex=True)
# rc('text', usetex=True)
# rc('mathtext', fontset ='dejavusans')
# rc('font', **{'family': 'sans-serif', 'sans-serif': ['DejaVu Sans'], 'size': 40})

plt.rcParams.update({
    "text.usetex": True,
    "font.family": "sans-serif",
    "font.serif": "Computer Modern Roman",
    "font.sans-serif": "Helvetica",
    "font.size": 42,
    'text.latex.preamble': r'\usepackage{sfmath}' # sans-serif for math
})

fig, ax = plt.subplots(figsize=(20, 12))   
ax.set_xlim(0, MAX_ITER)
ax.set_yscale('log')
ax.set_xlabel(r'Iteration $k$', fontsize=42)
ax.set_ylabel(r'Objective Error', fontsize=42)
ax.tick_params(axis='both', labelsize=42)
ax.yaxis.set_minor_locator(plt.LogLocator())
ax.spines.bottom.set_linewidth(2)
ax.spines.top.set_linewidth(2)
ax.spines.left.set_linewidth(2)
ax.spines.right.set_linewidth(2)
plt.grid(True, ls="--", color='0.65')


p1, = ax.plot(oe_logs['UDC_PEXTRA'], label='UDC-SingleRound', lw=3, c='xkcd:blue')
p2, = ax.plot(oe_logs['UDC_PGC'], lw=3, c='#00BFFF', alpha=0.5) # , label='UDC-PGC' sky blue
p3, = ax.plot(oe_logs['UDC_DPGA'], lw=3, c='#6699CC', alpha=0.4) # , label='UDC-DPGA' blue gray
p4, = ax.plot(oe_logs['UDC_DistADMM'], label='UDC-Dist.ADMM', ls='-', c='xkcd:green',lw=3)
p5, = ax.plot(oe_logs['ALT'], label='ALT', ls='-', lw=3, c='xkcd:orangered')
p6, = ax.plot(oe_logs['DPMM'], label='DPMM', ls='-.', c='orange', lw=3)
p7, = ax.plot(oe_logs['IPLUX'], label='IPLUX', ls='-.', c='xkcd:purple', lw=3)
p8, = ax.plot(oe_logs['dual subgradient'], label='dual subgradient', ls='-.', c='olive', lw=3)

ax.legend([(p1, p2, p3), p4, p5, p6, p7, p8], 
          ['UDC-SingleRound', 'UDC_DistADMM', 'ALT', 'DPMM', 
           'IPLUX', 'dual subgradient'], 
            handler_map={tuple: HandlerTuple(ndivide=None)},
            fontsize=40)
# ax.legend(fontsize=40)
fig.savefig(f'compare_obj_err.pdf', bbox_inches='tight', transparent=False)




fig, ax = plt.subplots(figsize=(20, 12))   
ax.set_xlim(0, MAX_ITER)
ax.set_yscale('log')
ax.set_xlabel(r'Iteration $k$', fontsize=42)
ax.set_ylabel(r'Constraint Violation', fontsize=42)
ax.tick_params(axis='both', labelsize=42)
ax.yaxis.set_minor_locator(plt.LogLocator())
ax.spines.bottom.set_linewidth(2)
ax.spines.top.set_linewidth(2)
ax.spines.left.set_linewidth(2)
ax.spines.right.set_linewidth(2)
plt.grid(True, ls="--", color='0.65')


p1, = ax.plot(cv_logs['UDC_PEXTRA'], label='UDC-SingleRound', lw=3, c='xkcd:blue')
p2, = ax.plot(cv_logs['UDC_PGC'], lw=3, c='#00BFFF', alpha=0.5) # , label='UDC-PGC'
p3, = ax.plot(cv_logs['UDC_DPGA'], lw=3, c='#6699CC', alpha=0.4) # , label='UDC-DPGA'
p4, = ax.plot(cv_logs['UDC_DistADMM'], label='UDC-Dist.ADMM', ls='-', c='xkcd:green',lw=3)
p5, = ax.plot(cv_logs['ALT'], label='ALT', ls='-', lw=3, c='xkcd:orangered')
p6, = ax.plot(cv_logs['DPMM'], label='DPMM', ls='-.', c='orange', lw=3)
p7, = ax.plot(cv_logs['IPLUX'], label='IPLUX', ls='-.', c='xkcd:purple', lw=3)
p8, = ax.plot(cv_logs['dual subgradient'], label='dual subgradient', ls='-.', c='olive', lw=3)

ax.legend([(p1, p2, p3), p4, p5, p6, p7, p8], 
          ['UDC-SingleRound', 'UDC_DistADMM', 'ALT', 'DPMM', 
           'IPLUX', 'dual subgradient'], 
            handler_map={tuple: HandlerTuple(ndivide=None)},
            fontsize=40)
fig.savefig(f'compare_cons_vio.pdf', bbox_inches='tight', transparent=False)




fig, ax = plt.subplots(figsize=(20, 12))   
ax.set_xlim(0, MAX_ITER)
ax.set_yscale('log')
ax.set_xlabel(r'Iteration $k$', fontsize=42)
ax.set_ylabel(r'$\Vert\mathit{x^k}-\mathit{x}^\star\Vert$', fontsize=42)
ax.tick_params(axis='both', labelsize=42)
ax.yaxis.set_minor_locator(plt.LogLocator())
ax.spines.bottom.set_linewidth(2)
ax.spines.top.set_linewidth(2)
ax.spines.left.set_linewidth(2)
ax.spines.right.set_linewidth(2)
plt.grid(True, ls="--", color='0.65')


ax.plot(xd_logs['UDC_PEXTRA'], label='UDC-PEXTRA', lw=3, c='orange')
# ax.plot(xd_logs['UDC_PGC'], label='UDC-PGC', ls='--', lw=3)
# ax.plot(xd_logs['UDC_DPGA'], label='UDC-DPGA', ls='--', lw=3)
ax.plot(xd_logs['UDC_DistADMM'], label='UDC-Dist.ADMM', ls='-', c='xkcd:green',lw=3)
ax.plot(xd_logs['ALT'], label='ALT', ls='-', lw=3, c='xkcd:orangered')
ax.plot(xd_logs['DPMM'], label='DPMM', ls='-.', c='xkcd:blue', lw=3)
ax.plot(xd_logs['IPLUX'], label='IPLUX', ls='-.', c='xkcd:purple', lw=3)
ax.plot(xd_logs['dual subgradient'], label='dual subgradient', ls='-.', c='olive', lw=3)

ax.legend(fontsize=40)
fig.savefig(f'compare_x_distance.pdf', bbox_inches='tight', transparent=False)

In [49]:

# ============================= trace in x1 x1 plane =========================
import matplotlib.pyplot as plt
import matplotlib.cm as cm

fig, ax = plt.subplots()     # Create a figure containing a single Axes.

delta = 0.05
x = np.arange(0.0, 1, delta)
y = np.arange(0.0, 1, delta)
X, Y = np.meshgrid(x, y)
F = X*alg.c[0] + Y*alg.c[1]
G = -alg.d[0]*np.log(1+X) - alg.d[1]*np.log(1+Y) + alg.b[0]*2/num_node

im = ax.imshow(F, interpolation='bilinear', origin='lower',
            cmap=cm.gray, extent=(0, 1, 0, 1))
CS_F = ax.contour(X, Y, F, 5)
ax.clabel(CS_F, fontsize=50)

# if there are only two nodes, we can polt the constraint
if num_node == 2:
    CS_G = ax.contour(X, Y, G, colors='k')
    ax.clabel(CS_G, fontsize=20)

x_log = np.array(alg.x_log)
x_avg_log = np.array(alg.x_avg_log)
ax.scatter(alg.x_star[0], alg.x_star[1], s=5000, c='r', marker='X', label='optimal')
ax.scatter(x_log[-1,0], x_log[-1,1], s=500, c='y', marker='^', label='last')
ax.scatter(x_log[:,0], x_log[:,1], label='iter', s=100, c='b', marker='^')  # Plot some data on the Axes.
ax.scatter(x_avg_log[:,0], x_avg_log[:,1], label='iter avg', s=100, marker='^')

ax.set_title('x position')
ax.set_xlabel('$x_0$')
ax.set_ylabel('$x_1$')
ax.legend()
fig.savefig('x.png')
plt.close()


AttributeError: 'UDC' object has no attribute 'x_log'